<a href="https://colab.research.google.com/github/MarwanKAsem/Codes/blob/main/ASR_using_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Whisper

In [3]:
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 139.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 47.6 MB

In [4]:
! pip install spela

In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


In [6]:
from datasets import load_dataset

dataset = load_dataset("distil-whisper/ami-ihm", "ihm")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.00k [00:00<?, ?B/s]

ami-ihm.py:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

The repository for distil-whisper/ami-ihm contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/distil-whisper/ami-ihm.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


EN2001a.tar.gz:   0%|          | 0.00/120M [00:00<?, ?B/s]

...

validation-transcription.csv:   0%|          | 0.00/953k [00:00<?, ?B/s]

test-transcription.csv:   0%|          | 0.00/913k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
# take the first sample of the validation set
dataset["validation"][0]

{'meeting_id': 'ES2011a',
 'id': 'AMI_ES2011a_H03_FEE044_0092784_0093052',
 'text': "BUT LIKE MOBILE PHONES HAVE SCREENS AND THEY'RE CHEAP",
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/39f9cd4b79885620930ee08a0d5d8e320b058663a767622d29d00bb4a90eff94/ES2011a/dev_ami_es2011a_h03_fee044_0092784_0093052.wav',
  'array': array([-1.22070312e-04, -9.15527344e-05, -9.15527344e-05, ...,
          1.52587891e-04,  3.05175781e-05,  1.83105469e-04]),
  'sampling_rate': 16000},
 'begin_time': 927.8400268554688,
 'end_time': 930.52001953125,
 'microphone_id': 'H03',
 'speaker_id': 'FEE044',
 'whisper_transcript': " But like mobile phones have screens and they're cheap."}

In [17]:
Train_Data=dataset["train"]
Test_Data=dataset["test"]

In [18]:
Train_Data[0]

{'meeting_id': 'EN2001a',
 'id': 'AMI_EN2001a_H04_MEO069_0330297_0330718',
 'text': 'IF YOU IF YOU S. S. H. AND THEY HAVE THIS BIG WARNING ABOUT DOING NOTHING AT ALL IN THE GATEWAY MACHINE',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/e9041db40e8bea1afc48f66ff890393f21d11a13f8eecd0e1d8fcdc96b1fde48/EN2001a/train_ami_en2001a_h04_meo069_0330297_0330718.wav',
  'array': array([ 0.00231934, -0.00183105, -0.00543213, ..., -0.00238037,
         -0.00244141, -0.00219727]),
  'sampling_rate': 16000},
 'begin_time': 3302.969970703125,
 'end_time': 3307.179931640625,
 'microphone_id': 'H04',
 'speaker_id': 'MEO069',
 'whisper_transcript': ' if you if you SSH in there this big warning about doing nothing at all in the gateway machine.'}

In [19]:
Train_Data = Train_Data.remove_columns(["id", "begin_time", "end_time", "microphone_id", "speaker_id","meeting_id"])

In [20]:
Test_Data = Test_Data.remove_columns(["id", "begin_time", "end_time", "microphone_id", "speaker_id","meeting_id"])

In [21]:
Train_Data[0]

{'text': 'IF YOU IF YOU S. S. H. AND THEY HAVE THIS BIG WARNING ABOUT DOING NOTHING AT ALL IN THE GATEWAY MACHINE',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/e9041db40e8bea1afc48f66ff890393f21d11a13f8eecd0e1d8fcdc96b1fde48/EN2001a/train_ami_en2001a_h04_meo069_0330297_0330718.wav',
  'array': array([ 0.00231934, -0.00183105, -0.00543213, ..., -0.00238037,
         -0.00244141, -0.00219727]),
  'sampling_rate': 16000},
 'whisper_transcript': ' if you if you SSH in there this big warning about doing nothing at all in the gateway machine.'}

In [22]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


In [23]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")


In [24]:
def prepare_dataset(batch):
    # Ensure the audio is correctly loaded
    audio = batch["audio"]

    # Compute log-Mel input features
    batch["input_features"] = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    # Encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch


In [25]:
Train_Data = Train_Data.map(
    prepare_dataset,
    remove_columns=["audio", "text"],
    num_proc=4
)


Map (num_proc=4):   0%|          | 0/108502 [00:00<?, ? examples/s]

RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.

In [ ]:
Train_Data = Train_Data.map(
    prepare_dataset,
    remove_columns=["audio", "text"],
)


Map:   0%|          | 0/108502 [00:00<?, ? examples/s]